In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

# Disable logs
tf.get_logger().setLevel('ERROR')

2022-11-04 19:45:31.126241: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-04 19:45:31.314597: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-04 19:45:31.377061: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-04 19:45:32.080759: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

In [2]:
# Settings
batch_size = 32
img_width = 320
img_height = 320

# Fetch dataset
train_ds: tf.data.Dataset
test_ds: tf.data.Dataset

(train_ds, test_ds), info = tfds.load('caltech_birds2011', split=['train', 'test'], 
                                      batch_size=batch_size, as_supervised=True, shuffle_files=True, with_info=True)

# Prepear dataset
def modify_images(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    
    image = tf.image.resize_with_pad(
        image,
        img_height, img_width,
        method='bilinear',
        antialias=False,
    )
    
    return image, label

train_ds = (
    train_ds
    .map(modify_images, num_parallel_calls=tf.data.AUTOTUNE)
    .cache()
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

test_ds = (
    test_ds
    .map(modify_images, num_parallel_calls=tf.data.AUTOTUNE)
    .cache()
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

num_classes = info.features['label'].num_classes

# Display dataset info
print("Train: {}, test: {}".format(len(train_ds), len(test_ds)))
print("Number of classes: {}".format(num_classes))

2022-11-04 19:45:33.556115: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-04 19:45:33.578103: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-04 19:45:33.578247: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-04 19:45:33.579092: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compi

Train: 188, test: 182
Number of classes: 200


In [3]:
from tensorflow import keras
from keras import layers

# Setup model
model = keras.Sequential([
    # Input
    layers.InputLayer(input_shape=(img_height, img_width, 3)),

    # Data augmentation
    layers.RandomContrast(factor=0.1),
    layers.RandomFlip(mode='horizontal'),
    layers.RandomRotation(factor=0.1),
    layers.RandomZoom(height_factor=0.1, width_factor=0.1),
    
    # Normalazation
    layers.Rescaling(scale=1./255),

    # Block 1
    layers.BatchNormalization(),
    layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(),

    # Block 2
    layers.BatchNormalization(),
    layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(),

    # Block 3
    layers.BatchNormalization(renorm=True),
    layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(),
    layers.Dropout(0.2),

    # Head
    layers.BatchNormalization(renorm=True),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes),
])

# Summarize model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_contrast (RandomCont  (None, 320, 320, 3)      0         
 rast)                                                           
                                                                 
 random_flip (RandomFlip)    (None, 320, 320, 3)       0         
                                                                 
 random_rotation (RandomRota  (None, 320, 320, 3)      0         
 tion)                                                           
                                                                 
 random_zoom (RandomZoom)    (None, 320, 320, 3)       0         
                                                                 
 rescaling (Rescaling)       (None, 320, 320, 3)       0         
                                                                 
 batch_normalization (BatchN  (None, 320, 320, 3)      1

In [ ]:
# Prepear model
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

# Train model
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=64,
)

# Save model
model.save("output/m2")

Epoch 1/64


2022-11-04 19:45:39.314021: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101
2022-11-04 19:45:42.362657: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


188/188 [==============================] - 90s 406ms/step - loss: 11.8475 - accuracy: 0.0028 - val_loss: 10342.4248 - val_accuracy: 0.0052
Epoch 2/64
188/188 [==============================] - 76s 403ms/step - loss: 34.6111 - accuracy: 0.0032 - val_loss: 543.7142 - val_accuracy: 0.0052
Epoch 3/64
188/188 [==============================] - 76s 402ms/step - loss: 7.2979 - accuracy: 0.0032 - val_loss: 6.0929 - val_accuracy: 0.0047
Epoch 4/64
188/188 [==============================] - 75s 400ms/step - loss: 5.6648 - accuracy: 0.0045 - val_loss: 5.3839 - val_accuracy: 0.0052
Epoch 5/64
188/188 [==============================] - 75s 399ms/step - loss: 5.5833 - accuracy: 0.0042 - val_loss: 5.3068 - val_accuracy: 0.0050
Epoch 6/64
188/188 [==============================] - 75s 400ms/step - loss: 6.2230 - accuracy: 0.0045 - val_loss: 5.3015 - val_accuracy: 0.0052
Epoch 7/64
188/188 [==============================] - 75s 399ms/step - loss: 5.3436 - accuracy: 0.0047 - val_loss: 5.3433 - val_accur

In [ ]:
# Plot learning curves
import pandas as pd

history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot()